### Extract text from Pdf documents and apply NER

In [3]:
import pdfplumber, os
import os.path as osp
import pandas as pd
import numpy as np
from pathlib import Path
from pdf_extract.services import file
from pdf_extract.config import global_config as glob
from pdf_extract.config import config
from pdf_extract.utils import utils
from importlib import reload
import fasttext

reload(glob)
reload(config)
reload(utils);

In [4]:
list_of_NEG_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'negatives'))
n_neg = len(list_of_NEG_docs)

list_of_POS_docs = os.listdir(osp.join(glob.UC_DATA_DIR, 'example_cvs', 'positives'))
n_pos = len(list_of_POS_docs)

print(n_pos)
print(n_neg)


13
15


In [5]:
# fname = 'CV4'

# i, page_objects, text = 0, {}, []
# # The open method returns an instance of the pdfplumber.PDF class.
# with pdfplumber.open(osp.join(glob.UC_DATA_DIR, f"example_cvs/{fname}.pdf")) as pdf:
#     while i < len(pdf.pages):
#         page = pdf.pages[i]
#         page_objects[str(i+1)] = page.extract_text(x_tolerance=1).split('\n')
#         #mystring = page_objects[str(i+1)].replace('\r', '').replace('\n', '')
#         text += page_objects[str(i+1)]
#         print(f"Page {i}")
#         #print(page.extract_text())
#         i += 1


In [6]:
#page_objects['1']

Read in all Pdfs in directory:

Postives only:

In [7]:
raw_corpus1 = pd.DataFrame(columns=['text', 'fname'])

for z, fname in enumerate(list_of_POS_docs):
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"positives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            #print(pdf.metadata)
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            #print(f"Page {i}")
            #print(page.extract_text())
            i += 1
    
    raw_corpus1.loc[z] = [text,fname]  
raw_corpus1['label'] = 'positive'

Negatives only:

In [8]:
raw_corpus2 = pd.DataFrame(columns=['text', 'fname'])

for z, fname in enumerate(list_of_NEG_docs):
    i, page_objects, text = 0, {}, ""
    # The open method returns an instance of the pdfplumber.PDF class.
    with pdfplumber.open(osp.join(glob.UC_DATA_DIR, "example_cvs",f"negatives/{fname}")) as pdf:
        while i < len(pdf.pages):
            page = pdf.pages[i]
            page_objects[str(i+1)] = page.extract_text(x_tolerance=1, y_tolerance=3) #.split('\n')
            text += page_objects[str(i+1)]
            i += 1
    
    raw_corpus2.loc[z] = [text,fname]   
raw_corpus2['label'] = 'negative'

In [9]:
raw_corpus = pd.concat([raw_corpus1,raw_corpus2], axis=0, ignore_index=True)
#raw_corpus

In [10]:
#page_objects

#page.extract_words()

In [11]:
#page_instance = page_objects['1']

In [12]:
#pdf.metadata
#page_instance

Preprocess text corpus:

In [13]:
from pdf_extract.resources import preprocessor as preproc

reload(preproc)

# Preprocess corpus:
cleaner = preproc.clean_text(language='english', lemma = False, stem = False)

X_raw = raw_corpus['text'].copy()

# Full sample
X_cleaned = cleaner.fit_transform(X_raw)   

#X = raw_corpus['text']

#X_cl = cleaner.fit_transform(X_raw)

docs = X_cleaned.tolist()    

Using english language.
Using 179 stop words.
Added 8 stopword(s).
Added 13 stopword(s).
Adding custom stop words...
Setting to lower cases.
Removing whitespaces.
Applying word tokenizer.
Removing custom stopwords.
Removing punctuations.
Removing numbers.
Removing digits.
Removing non-alphabetic characters.
Removing short tokens.
Finished preprocessing!


[nltk_data] Downloading package punkt to /home/alexv84/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexv84/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
raw_corpus.head()

,text,fname,label
0,Ghassen BEN MAKHLOUF\nStudent | M.Sc in Comput...,cv17.pdf,positive
1,Tobias Watzel\nZUSAMMENFASSUNG\nExperte für Ma...,cv20.pdf,positive
2,Gabriel Kordon \n \n \n+49 157 31683...,cv18.pdf,positive
3,CEYDA AKBULUT\nM . S c . D a t a E n t h u...,cv15.pdf,positive
4,D R . A L E X A N D E R V O S S E L E R\nperso...,cv26.pdf,positive


In [22]:
combine_fct = lambda x: (x.label == 'positive')*1

raw_corpus['text_cleaned'] = X_cleaned
raw_corpus['target'] = raw_corpus.apply(combine_fct, axis = 1)

X = raw_corpus[['target', 'text_cleaned', 'text']].reset_index(drop=True)   # keep label in X for txt-file below, will not be used directly in ft
X.target = X.target.astype(str)
X.head()

,target,text_cleaned,text
0,1,ghassen makhlouf student computer science link...,Ghassen BEN MAKHLOUF\nStudent | M.Sc in Comput...
1,1,tobias watzel zusammenfassung experte machine ...,Tobias Watzel\nZUSAMMENFASSUNG\nExperte für Ma...
2,1,gabriel kordon kordon mangfallstraße rosenheim...,Gabriel Kordon \n \n \n+49 157 31683...
3,1,ceyda akbulut phone email akbulut gmail linked...,CEYDA AKBULUT\nM . S c . D a t a E n t h u...
4,1,personal information born sindelﬁngen germany ...,D R . A L E X A N D E R V O S S E L E R\nperso...


In [19]:
from spacy.tokens import DocBin
import spacy
from tqdm import tqdm

nlp = spacy.load("de_core_news_lg") 
#nlp = spacy.blank('en')              # blank model; to train from scratch on own corpus

nlp.pipe_names

['tok2vec',
 'tagger',
 'morphologizer',
 'parser',
 'lemmatizer',
 'attribute_ruler',
 'ner']

In [20]:
from typing import Set, List, Tuple
from spacy.tokens import DocBin

def make_docs(data: List[Tuple[str, str]], target_file: str, cats: Set[str]):
    
    docs = DocBin()
    for doc, label in nlp.pipe(data, as_tuples=True):
        for cat in cats:
            doc.cats[cat] = 1 if cat == label else 0
        docs.add(doc)
    docs.to_disk(os.path.join(glob.UC_DATA_DIR, target_file))
    print(os.path.join(glob.UC_DATA_DIR, target_file))
    return docs

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X["text_cleaned"].values, X["target"].values, test_size=0.3)

print(X_train.shape)
print(X_test.shape)

cats = ["0","1"]

make_docs(list(zip(X_train, y_train)), "train.spacy", cats=cats)
make_docs(list(zip(X_test, y_test)), "valid.spacy", cats=cats)

(19,)
(9,)
/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/train.spacy
/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/valid.spacy


In [24]:
!python -m spacy init fill-config /home/alexv84/Documents/Arbeit/Allianz/AZVers/data/base_config_textclf.cfg /home/alexv84/Documents/Arbeit/Allianz/AZVers/data/config.cfg   

✔ Auto-filled config with all values
✔ Saved config
/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [25]:
from spacy.cli.train import train

train(config_path = f"{glob.UC_DATA_DIR}/config.cfg",
        output_path = f"{glob.UC_DATA_DIR}/output",
        overrides={"paths.train": f"{glob.UC_DATA_DIR}/train.spacy",
                "paths.dev": f"{glob.UC_DATA_DIR}/valid.spacy",
    },
)

print("Training done!")

ℹ Saving to output directory:
/home/alexv84/Documents/Arbeit/Allianz/AZVers/data/output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ------------  ----------  ------
  0       0          0.00          0.25       64.94    0.65
 10     200          7.57         22.78       64.94    0.65
 21     400         62.92          6.69       58.46    0.58
 31     600         14.36          4.55       75.00    0.75
 42     800          0.00          0.00       75.00    0.75
 52    1000          0.00          0.00       75.00    0.75
 63    1200          0.00          0.00       75.00    0.75
 73    1400          0.00          0.00       75.00    0.75
 84    1600          0.00          0.00       75.00    0

In [ ]:
# Use best model fit:
trained_nlp = spacy.load(f"{glob.UC_DATA_DIR}/output/model-best") 


In [ ]:
id = 2

text = X['text_cleaned'][id]
print(text)
print(X['target'][id])

# Perform the trained pipeline on this text
doc = trained_nlp(text)

doc.cats

In [ ]:
doc_from_pretrained = nlp(text)

print("entities", doc_from_pretrained.ents)
print("sentences", list(doc_from_pretrained.sents))
print("classification", doc_from_pretrained.cats)    # empty -> add trained_nlp pipeline!

In [ ]:
nlp = spacy.load("de_core_news_lg", exclude=["textcat"])
nlp.add_pipe("textcat", source = trained_nlp)

In [41]:
spacy.explain("ORG")

'Companies, agencies, institutions, etc.'

In [42]:
spacy.explain("GPE")

'Countries, cities, states'

In [44]:
#displacy.render(text1,style="ent",jupyter=True)

In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [28]:
URL="https://www.zeebiz.com/markets/currency/news-cryptocurrency-news-today-june-12-bitcoin-dogecoin-shiba-inu-and-other-top-coins-prices-and-all-latest-updates-158490"

In [29]:
html_content = requests.get(URL).text

In [30]:
soup = BeautifulSoup(html_content, "lxml")

In [31]:
body=soup.body.text

In [32]:
body= body.replace('\n', ' ')
#body= body.replace('t', ' ')
#body= body.replace('r', ' ')
#body= body.replace('xa0', ' ')
#body=re.sub(r'[^ws]', '', body)

In [148]:
#body

In [149]:
text3= NER(body)